In [2]:
import pandas as pd

/Users/aldemarogonzalez/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/2r/b60vp1dj2zn0rg5yg_gyy0zr0000gq/T/ipykernel_12452/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# organization_path = '../Dataset/bulk_export/organizations.csv' 
organization_path = '../Dataset/filtered_data/organizations_filtered_by_date_and_roles.csv' 
organization_data = pd.read_csv(organization_path)

# Step 2: Manipulate or analyze the data as needed
# For example, you can print the first few rows of the DataFrame using .head()
# print("First 5 rows of the DataFrame:")
# print(organization_data.head())

# Or you can perform some basic analysis, such as getting summary statistics using .describe()
# print("\nSummary statistics of the DataFrame:")
# print(organization_data.describe())

organization_data_num_rows = organization_data.shape[0]
print("Number of Organization in the Organization.csv:", organization_data_num_rows)
organization_data_num_columns = organization_data.shape[1] 
print("Number of columns in the Organization.csv:", organization_data_num_columns)
organization_data_column_names = organization_data.columns
print("Column names in the Organization.csv:")
for name in organization_data_column_names:
    print(name)

# Filtering the companies

1 - I filtered companies using their creation date

In [ ]:
from datetime import datetime

# Define the warm up window 
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # Start Day: 1 of January of 2015
end_date = datetime.strptime('2018-12-31', '%Y-%m-%d') # End Day: 31 of December of 2018

In [ ]:
import csv

with open('../Dataset/bulk_export/organizations.csv', 'r') as f, open('../Dataset/filtered_data/organizations_filtered_by_date.csv', 'w', newline='') as f_out:
    reader = csv.reader(f)
    writer = csv.writer(f_out)

    header = next(reader)
    writer.writerow(header)

    date_index = header.index('founded_on')

    # Read the rest of the csv and write rows where the date is within the range
    for row in reader:
        date_str = row[date_index]
        if date_str:  # Check if the date string is not empty
            try:
                date = datetime.strptime(date_str, '%Y-%m-%d')
                if start_date <= date <= end_date:
                    writer.writerow(row)
            except ValueError:
                print("Error: Unable to parse date string '{}'".format(date_str))
        else:
            print("Warning: Empty date string encountered in row {}".format(row))

Looking at the dataset, we see that there is a column that also let us differentiate between an Companies and Investment firms in the domain column

In [ ]:
with open('../Dataset/filtered_data/organizations_filtered_by_date.csv', 'r') as f, open('../Dataset/filtered_data/organizations_filtered_by_date_and_roles.csv', 'w', newline='') as f_out:
    reader = csv.reader(f)
    writer = csv.writer(f_out)

    header = next(reader)  # This gets the header row
    writer.writerow(header)  # Write the header to the new file

    # Find the index of the date column (replace 'date_column' with your column's name)
    roles_index = header.index('roles')

    # Read the rest of the csv and write rows where the date is within the range
    for row in reader:
        roles_string = row[roles_index]
        if roles_string:  # Check if the date string is not empty
            try:
                if roles_string == "company":
                    writer.writerow(row)
            except ValueError:
                print("Error: Unable to parse string string '{}'".format(roles_string))
        else:
            print("Warning: Empty string string encountered in row {}".format(row))


Clean the database by dropping the columns that are not relevant for the study

In [ ]:
# Specify the columns to drop
columns_to_drop = ['type', 
    'cb_url', 
    'rank', 
    'created_at', 
    'updated_at', 
    'legal_name', 
    'roles', 
    'cb_url', 
    'rank', 
    'created_at', 
    'updated_at', 
    'legal_name', 
    'roles', 
    'email', 
    'phone', 
    'logo_url', 
    'alias1', 
    'alias2', 
    'alias3', 
    'primary_role', 
    'num_exits']

# Drop the specified columns
organizations_data = organization_data.drop(columns=columns_to_drop)
organizations_data.to_csv('../Dataset/filtered_data/filtered_organization.csv', index=False)

### Now I merge the organizations with their funding data, IPO data, closure data to further filtering by IPO/Acquisition/Closure

In [ ]:
funding_path = '../Dataset/bulk_export/funding_rounds.csv' 

funding_data = pd.read_csv(funding_path)

In [ ]:
organizations_path = '../Dataset/filtered_data/filtered_organization.csv'

organizations_data = pd.read_csv(organizations_path)

In [ ]:
merged_df = organizations_data.merge(funding_data, left_on='uuid', right_on='org_uuid', how="left")

In [ ]:
merged_df.to_csv('../Dataset/filtered_data/filtered_organization_merged.csv', index=False)

In [ ]:
merged_df_path = '../Dataset/filtered_data/filtered_organization_merged.csv' 

merged_df = pd.read_csv(merged_df_path)

First - I removed the ones closed during the Warmup window 

In [ ]:
merged_df['closed_on'] = pd.to_datetime(merged_df['closed_on'])
filtered_close_merged_df = merged_df[(merged_df['closed_on'] < start_date) | (merged_df['closed_on'] > end_date) | (merged_df['closed_on'].isna())]


Now I removed the ones acquired during the Warmup window 

In [ ]:
acquisitions_path = '../Dataset/bulk_export/acquisitions.csv'  # Replace 'your_file.csv' with the actual path to your CSV file
acquisitions_data = pd.read_csv(acquisitions_path)
acquisitions_data_num_rows = acquisitions_data.shape[0] 
print("Number of acquisitions in the acquisitions.csv:", acquisitions_data_num_rows)
acquisitions_data_num_columns = acquisitions_data.shape[1] 
print("Number of columns in the acquisitions.csv:", acquisitions_data_num_columns)
acquisitions_data_column_names = acquisitions_data.columns
print("Column names in the acquisitions.csv:")
for name in acquisitions_data_column_names:
    print(name)

In [ ]:
acquired_df =  filtered_close_merged_df.merge(acquisitions_data, left_on='uuid_x', right_on='acquiree_uuid', how="left")

In [ ]:
acquired_df.to_csv('../Dataset/filtered_data/acquired_df.csv', index=False)

In [ ]:
acquired_df['acquired_on'] = pd.to_datetime(acquired_df['acquired_on'])
filtered_close_acquired_df = acquired_df[(acquired_df['acquired_on'] < start_date) | (acquired_df['acquired_on'] > end_date) | (acquired_df['acquired_on'].isna())]


In [ ]:
filtered_close_acquired_df.to_csv('../Dataset/filtered_data/filtered_close_acquired_df.csv', index=False)

Now I removed the ones that IPO during the Warmup window 

In [ ]:
ipos_path = '../Dataset/bulk_export/ipos.csv'  # Replace 'your_file.csv' with the actual path to your CSV file
ipos_data = pd.read_csv(ipos_path)
ipos_data_num_rows = ipos_data.shape[0] 
print("Number of ipos in the ipos.csv:", ipos_data_num_rows)
ipos_data_num_columns = ipos_data.shape[1] 
print("Number of columns in the ipos.csv:", ipos_data_num_columns)
ipos_data_column_names = ipos_data.columns
print("Column names in the ipos.csv:")
for name in ipos_data_column_names:
    print(name)

In [ ]:
# Rename the columns before merging to ensure they have unique names
filtered_close_acquired_df.rename(columns={'uuid_x': 'uuid_org'}, inplace=True)
filtered_close_acquired_df.rename(columns={'name_x': 'name_org'}, inplace=True)
filtered_close_acquired_df.rename(columns={'permalink_x': 'permalink_org'}, inplace=True)

ipos_data.rename(columns={'org_uuid': 'uuid_ipos'}, inplace=True)


In [ ]:
ipos_df =  filtered_close_acquired_df.merge(ipos_data, left_on='uuid_org', right_on='uuid_ipos', how="left")

In [ ]:
ipos_df.to_csv('../Dataset/filtered_data/ipos_df.csv', index=False)

In [ ]:
ipos_df['went_public_on'] = pd.to_datetime(ipos_df['went_public_on'])
filtered_close_acquired_ipos_df = ipos_df[(ipos_df['went_public_on'] < start_date) | (ipos_df['went_public_on'] > end_date) | (ipos_df['went_public_on'].isna())]


In [ ]:
filtered_close_acquired_ipos_df.to_csv('../Dataset/filtered_data/filtered_close_acquired_ipos_df.csv', index=False)

## I filter the companies which closed a funding round above Series B or above, which is not interesting for Plug and Play investments


In [ ]:
# Define the series that are Series B or above
series_b_or_above = ['series_b', 'series_c', 'series_d', 'series_e', 'series_f', 'series_g']

# Filter the DataFrame to exclude organizations that have a series B or above in their investment_type

organizations_to_remove = filtered_close_acquired_ipos_df[filtered_close_acquired_ipos_df['investment_type'].isin(series_b_or_above)]['org_uuid'].unique()

In [ ]:
filtered_series_df = filtered_close_acquired_ipos_df[~filtered_close_acquired_ipos_df['org_uuid'].isin(organizations_to_remove)]


In [ ]:
filtered_series_df.to_csv('../Dataset/filtered_data/filtered_series_df.csv', index=False)

-------------

In [ ]:
filtered_df_path = '../Dataset/filtered_data/filtered_series_df.csv'

filtered_data = pd.read_csv(filtered_df_path)

In [ ]:
#Some companies are duplicate because they had a few rounds, this is for taking them away
unique_filtered = filtered_data.drop_duplicates(subset=['uuid_org'], keep='first')


In [ ]:
num_rows = unique_filtered.shape[0]

print(f"There is {num_rows} companies.")

Now we need to clean the data set by removing unwanted columns

In [ ]:
column_names = unique_filtered.columns.tolist()

# Join the column names with commas
column_names_str = '\', \''.join(column_names)

# Print the column names
print(column_names_str)

In [ ]:
columns_to_drop = ['uuid_y', 'name_y', 'type_x', 'permalink_y', 'cb_url_x', 'rank_x', 'created_at_x', 'updated_at_x', 'country_code_y', 'state_code_y', 'region_y', 'city_y', 'investment_type', 'announced_on', 'raised_amount_usd', 'raised_amount', 'raised_amount_currency_code', 'post_money_valuation_usd', 'post_money_valuation', 'post_money_valuation_currency_code', 'investor_count', 'org_uuid', 'org_name_x', 'lead_investor_uuids', 'uuid_x', 'name_x', 'type_y', 'permalink_x', 'cb_url_y', 'rank_y', 'created_at_y', 'updated_at_y', 'acquiree_uuid', 'acquiree_name', 'acquiree_cb_url', 'acquiree_country_code', 'acquiree_state_code', 'acquiree_region', 'acquiree_city', 'acquirer_uuid', 'acquirer_name', 'acquirer_cb_url', 'acquirer_country_code', 'acquirer_state_code', 'acquirer_region', 'acquirer_city', 'acquisition_type', 'acquired_on', 'price_usd', 'price', 'price_currency_code', 'uuid_y.1', 'name_y.1', 'type', 'permalink_y.1', 'cb_url', 'rank', 'created_at', 'updated_at', 'uuid_ipos', 'org_name_y', 'org_cb_url', 'country_code', 'state_code', 'region', 'city', 'stock_exchange_symbol', 'stock_symbol', 'went_public_on', 'share_price_usd', 'share_price', 'share_price_currency_code', 'valuation_price_usd', 'valuation_price', 'valuation_price_currency_code', 'money_raised_usd', 'money_raised', 'money_raised_currency_code']


In [ ]:
unique_filtered = unique_filtered.drop(columns=columns_to_drop)

In [14]:
column_names = unique_filtered.columns.tolist()

# Join the column names with commas
column_names_str = '\', \''.join(column_names)

# Print the column names
print(column_names_str)

uuid_org', 'name_org', 'permalink_org', 'domain', 'homepage_url', 'country_code_x', 'state_code_x', 'region_x', 'city_x', 'address', 'postal_code', 'status', 'short_description', 'category_list', 'category_groups_list', 'num_funding_rounds', 'total_funding_usd', 'total_funding', 'total_funding_currency_code', 'founded_on', 'last_funding_on', 'closed_on', 'employee_count', 'facebook_url', 'linkedin_url', 'twitter_url


## Now we need to add the predictor variables 

And also delete some attributes that I do not consider relevant

In [15]:
unique_filtered.rename(columns={'country_code_x': 'country_code'}, inplace=True)
unique_filtered.rename(columns={'state_code_x': 'state_code'}, inplace=True)
unique_filtered.rename(columns={'region_x': 'region'}, inplace=True)
unique_filtered.rename(columns={'city_x': 'city'}, inplace=True)

In [16]:
unique_filtered.to_csv('../Dataset/filtered_data/unique_filtered.csv', index=False)

In [21]:
unique_filtered = unique_filtered.drop(columns='employee_count')

Nota para alde: creo que deberia quitar las compañias que tengan al menos 1 funding round pero no diga la cantidad... No lo se


## Company Information

we included the company age in months (age_months) at the beginning of the simulation (ts) and we added variables that measure the presence of the company in social media networks. 

These binary variables indicate whether the company registered in Crunchbase its con- tact information (has_email and has_phone) or a Facebook (has_facebook_url), Twitter (has_twitter_url) or Linkedin (has_linkedin_url) account.

In [7]:
#Beginning of the simulation period

from datetime import datetime

# Define the warm up window 
simulation_start_date = datetime.strptime('2019-01-01', '%Y-%m-%d') # Start Day: 1 of January of 2019
simulation_end_date = datetime.strptime('2022-12-31', '%Y-%m-%d') # End Day: 31 of December of 2022

In [23]:
import math

unique_filtered['founded_on'] = pd.to_datetime(unique_filtered['founded_on'])

unique_filtered['age_months'] = ((simulation_start_date - unique_filtered['founded_on']).dt.days / 30).apply(math.ceil)


In [24]:
# Define a function to check if the URL is present
def has_url(url):
    return 1 if pd.notnull(url) else 0

# Convert URLs into binary variables
unique_filtered['has_facebook_url'] = unique_filtered['facebook_url'].apply(has_url)
unique_filtered['has_twitter_url'] = unique_filtered['twitter_url'].apply(has_url)
unique_filtered['has_linkedin_url'] = unique_filtered['linkedin_url'].apply(has_url)

In [25]:
unique_filtered.to_csv('../Dataset/filtered_data/unique_filtered.csv', index=False)

## Funding information

The variables in this category summarize the funding events occurred in a company during the Warmup window. The availability of a temporal series of funding round events in Crunchbase allowed us to synthesize information about:

- Number of funding rounds that the company achieved before ts (round_count) and the total amount raised in those rounds(raised_amount_usd).

In [3]:
funding_rounds_path = '../Dataset/bulk_export/funding_rounds.csv' 
funding_rounds_data = pd.read_csv(funding_rounds_path)

# Step 2: Manipulate or analyze the data as needed
# For example, you can print the first few rows of the DataFrame using .head()
# print("First 5 rows of the DataFrame:")
# print(funding_rounds_data.head())

# Or you can perform some basic analysis, such as getting summary statistics using .describe()
# print("\nSummary statistics of the DataFrame:")
# print(funding_rounds_data.describe())

funding_rounds_data_num_rows = funding_rounds_data.shape[0]
print("Number of funding_rounds in the funding_rounds.csv:", funding_rounds_data_num_rows)
funding_rounds_data_num_columns = funding_rounds_data.shape[1] 
print("Number of columns in the funding_rounds.csv:", funding_rounds_data_num_columns)
funding_rounds_data_column_names = funding_rounds_data.columns
print("Column names in the funding_rounds.csv:")
for name in funding_rounds_data_column_names:
    print(name)

Number of funding_rounds in the funding_rounds.csv: 631092
Number of columns in the funding_rounds.csv: 24
Column names in the funding_rounds.csv:
uuid
name
type
permalink
cb_url
rank
created_at
updated_at
country_code
state_code
region
city
investment_type
announced_on
raised_amount_usd
raised_amount
raised_amount_currency_code
post_money_valuation_usd
post_money_valuation
post_money_valuation_currency_code
investor_count
org_uuid
org_name
lead_investor_uuids


In [4]:
unique_filtered = pd.read_csv('../Dataset/filtered_data/unique_filtered.csv')

merged_unique_funding_df = unique_filtered.merge(funding_rounds_data, left_on='uuid_org', right_on='org_uuid', how="left")
num_rows = merged_unique_funding_df.shape[0]

print(f"There is {num_rows} companies.")

There is 449051 companies.


In [5]:
merged_unique_funding_df_column_names = merged_unique_funding_df.columns
print("Column names in the funding_rounds.csv:")
for name in merged_unique_funding_df_column_names:
    print(name)

Column names in the funding_rounds.csv:
uuid_org
name_org
permalink_org
domain
homepage_url
country_code_x
state_code_x
region_x
city_x
address
postal_code
status
short_description
category_list
category_groups_list
num_funding_rounds
total_funding_usd
total_funding
total_funding_currency_code
founded_on
last_funding_on
closed_on
facebook_url
linkedin_url
twitter_url
age_months
has_facebook_url
has_twitter_url
has_linkedin_url
uuid
name
type
permalink
cb_url
rank
created_at
updated_at
country_code_y
state_code_y
region_y
city_y
investment_type
announced_on
raised_amount_usd
raised_amount
raised_amount_currency_code
post_money_valuation_usd
post_money_valuation
post_money_valuation_currency_code
investor_count
org_uuid
org_name
lead_investor_uuids


In [8]:
# Convert 'announced_on' column to datetime if it's not already
merged_unique_funding_df['announced_on'] = pd.to_datetime(merged_unique_funding_df['announced_on'])

# Filter the funding rounds that occurred before ts
funding_before_ts = merged_unique_funding_df[merged_unique_funding_df['announced_on'] < simulation_start_date]

# Count the number of funding rounds for each company
round_count = funding_before_ts.groupby('uuid_org').size().rename('round_count')

# Sum up the raised amount in those rounds for each company
raised_amount_usd = funding_before_ts.groupby('uuid_org')['raised_amount_usd'].sum().rename('raised_amount_usd')

# Merge round count and raised amount into merged_unique_funding_df DataFrame
merged_unique_funding_df = pd.merge(merged_unique_funding_df, round_count, left_on='uuid_org', right_index=True, how='left')
merged_unique_funding_df = pd.merge(merged_unique_funding_df, raised_amount_usd, left_on='uuid_org', right_index=True, how='left')

# Replace NaN values with 0 for companies with no data
merged_unique_funding_df.fillna(0, inplace=True)

/var/folders/2r/b60vp1dj2zn0rg5yg_gyy0zr0000gq/T/ipykernel_12452/1756900613.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  merged_unique_funding_df.fillna(0, inplace=True)


- Data about the last funding round in Warmup window: funding round type (last_round_investment_type), amount raised(last_round_raised_amount_usd), company valuation after this funding round(last_round_post_money_valuation) and time lapsed, in months, between the beginning of the simulation (ts) and when this funding round occurred(last_round_timelapse_months)

In [12]:
import math

# 2. Find the last funding round for each company within the Warmup window
last_round_warmup = funding_before_ts.groupby('uuid_org').last()

# 3. Extract the required data from the last funding round for each company
last_round_investment_type = last_round_warmup['investment_type']
last_round_raised_amount_usd = last_round_warmup['raised_amount_usd']
last_round_post_money_valuation = last_round_warmup['post_money_valuation_usd']

# 4. Calculate the time lapse in months between simulation start date and the last funding round
last_round_timelapse_months = ((simulation_start_date - last_round_warmup['announced_on']).dt.days / 30).apply(math.ceil).astype(int)

# Add extracted data to merged_unique_funding_df DataFrame
merged_unique_funding_df['last_round_investment_type'] = last_round_investment_type
merged_unique_funding_df['last_round_raised_amount_usd'] = last_round_raised_amount_usd
merged_unique_funding_df['last_round_post_money_valuation'] = last_round_post_money_valuation
merged_unique_funding_df['last_round_timelapse_months'] = last_round_timelapse_months

# Replace NaN values with 0 for companies with no data
merged_unique_funding_df.fillna(0, inplace=True)

/var/folders/2r/b60vp1dj2zn0rg5yg_gyy0zr0000gq/T/ipykernel_12452/3126441978.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_unique_funding_df.fillna(0, inplace=True)


- Number of (unique) investors who participated in the funding rounds during the Warmup window (investor_count) and specifically, in the last funding (last_round_investor_count).

In [14]:
investments_path = '../Dataset/bulk_export/investments.csv' 
investments_data = pd.read_csv(investments_path)

# Step 2: Manipulate or analyze the data as needed
# For example, you can print the first few rows of the DataFrame using .head()
# print("First 5 rows of the DataFrame:")
# print(investments_data.head())

# Or you can perform some basic analysis, such as getting summary statistics using .describe()
# print("\nSummary statistics of the DataFrame:")
# print(investments_data.describe())

investments_data_num_rows = investments_data.shape[0]
print("Number of investments in the investments.csv:", investments_data_num_rows)
investments_data_num_columns = investments_data.shape[1] 
print("Number of columns in the investments.csv:", investments_data_num_columns)
investments_data_column_names = investments_data.columns
print("Column names in the investments.csv:")
for name in investments_data_column_names:
    print(name)

Number of investments in the investments.csv: 1027342
Number of columns in the investments.csv: 14
Column names in the investments.csv:
uuid
name
type
permalink
cb_url
rank
created_at
updated_at
funding_round_uuid
funding_round_name
investor_uuid
investor_name
investor_type
is_lead_investor


In [16]:
investments_filtered =  merged_unique_funding_df.merge(investments_data, right_on='funding_round_uuid', left_on='uuid', how="left")

In [17]:
investors_path = '../Dataset/bulk_export/investors.csv' 
investors_data = pd.read_csv(investors_path)

# Step 2: Manipulate or analyze the data as needed
# For example, you can print the first few rows of the DataFrame using .head()
# print("First 5 rows of the DataFrame:")
# print(investors_data.head())

# Or you can perform some basic analysis, such as getting summary statistics using .describe()
# print("\nSummary statistics of the DataFrame:")
# print(investors_data.describe())

investors_data_num_rows = investors_data.shape[0]
print("Number of investors in the investors.csv:", investors_data_num_rows)
investors_data_num_columns = investors_data.shape[1] 
print("Number of columns in the investors.csv:", investors_data_num_columns)
investors_data_column_names = investors_data.columns
print("Column names in the investors.csv:")
for name in investors_data_column_names:
    print(name)

Number of investors in the investors.csv: 274366
Number of columns in the investors.csv: 25
Column names in the investors.csv:
uuid
name
type
permalink
cb_url
rank
created_at
updated_at
roles
domain
country_code
state_code
region
city
investor_types
investment_count
total_funding_usd
total_funding
total_funding_currency_code
founded_on
closed_on
facebook_url
linkedin_url
twitter_url
logo_url


In [18]:
investments_filtered_investors =  investments_filtered.merge(investors_data, right_on='uuid', left_on='investor_uuid', how="left")

- Qualitative information about the investors. We defined a category of renowned investors as the investment companies registered in Crunchbase and created variables for the number of unique renowned investors who participated in the funding rounds during the Warmup window (known_investor_count) and, specifically, in the last funding (last_round_known_investor_count).

## Founders information
The last category comprises information about the people who founded a company. In addition to the number of founder(founders_count), we synthesized new variables that provide information about the heterogeneity of the founders according to their origin number of different countries where the founders come from(founders_dif_country_count)– and gender –number of male
(founders_male_count) and female (founders_female_count)founders.

Previous studies highlighted the importance of having a college education when building a new company. Crunchbase provides information about the education of most of the founders. However, the way this information is stored (in a free-form text) hinders the synthesis of qualitative variables about the education received by company founders.

After revisiting the information contained in Crunchbase and observing that most of the education entries refer to higher education, we decided to synthesize quantitative variables about the total number of degrees obtained by company founders (founders_degree_count_total), as well as the maximum (founders_degree_count_max) and the average number of degrees (founders_degree_count_mean) among them.

The sparsity problem is evident also in this category, as most of the companies do not have information about their founders or their education. In this case, we consider the absence of data as useful information and consider 0, where it corresponds. It means that the company has not updated the information about the founders in Crunchbase